# Central Carbon Genes
This notebook gets the genes involved in the reactions of central carbon metabolism

### Load imports

In [1]:
import cobra
import pandas as pd

### Load model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_2.json")
model

Name,model
Memory address,106ab2a70
Number of metabolites,1121
Number of reactions,1348
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Load 13C-MFA to GSM mapping for the 3 carbon sources

In [3]:
# load the 13C MFA data
glucose_13c_mfa_df = pd.read_csv('../data/13c_mfa/glucose_13c_mfa.csv')
oleic_acid_13c_mfa_df = pd.read_csv('../data/13c_mfa/oleic_acid_900_13c_mfa.csv')
glycerol_13c_mfa_df = pd.read_csv('../data/13c_mfa/glycerol_13c_mfa.csv')

# get oleic acid uptake and glycerol uptake reactions
oleic_acid_uptake_row = oleic_acid_13c_mfa_df[oleic_acid_13c_mfa_df['pathway'] == 'substrate_uptake']
glycerol_uptake_row = glycerol_13c_mfa_df[glycerol_13c_mfa_df['pathway'] == 'substrate_uptake']

# add oleic acid uptake and glycerol uptake reaction rows to the top of glucose_13c_mfa_df
central_carbon_df = pd.concat([oleic_acid_uptake_row, glycerol_uptake_row, glucose_13c_mfa_df])

# drop these columns: 13C_mfa_flux	StdErr	13C_mfa_flux_95%_lower_bound	13C_mfa_flux_95%_upper_bound
central_carbon_df = central_carbon_df.drop(columns=['13C_mfa_flux', 'StdErr', '13C_mfa_flux_95%_lower_bound', '13C_mfa_flux_95%_upper_bound'])

central_carbon_df

,pathway,compartment,Equation,reaction_ids
0,substrate_uptake,cytosol,Oleic Acid -> 9 AC,OCDCEAt
0,substrate_uptake,cytosol,Glycerol -> GLYC,reverse_GLYCt
0,substrate_uptake,cytosol,Glucose + ATP -> G6P,reverse_EX_glc(e)
1,emp,cytosol,G6P <-> F6P,PGI
2,emp,cytosol,F6P + ATP <-> FBP,PFK or reverse_FBP
...,...,...,...,...
74,transport,extracellular,CO2 -> CO2_EX,reverse_CO2t
75,biomass_formation,cytosol,0.247*ALA + 0.146*ARG + 0.1165*ASN + 0.165*ASP...,biomass_C
76,energy_metabolism,cytosol,ATP -> ATP_maintenance,ATPM
77,energy_metabolism,cytosol,NADH -> NADPH,NaN


### Connect reaction to GSM genes

In [4]:
list_of_genes = []

for _, row in central_carbon_df.iterrows():
    reaction = row['Equation']
    reaction_ids = row['reaction_ids']

    # before splitting reaction ids, remove ensure that it is a string
    reaction_ids = str(reaction_ids)

    # split reaction ids by space
    reaction_ids = reaction_ids.split(' ')

    # remove 'nan', 'and', and 'or' from reaction ids
    reaction_ids = [x for x in reaction_ids if x != 'nan' and x != 'and' and x != 'or']

    genes_for_reaction = []

    for reaction_id in reaction_ids:
        if 'reverse_' in reaction_id:
            reaction_id = reaction_id.replace('reverse_', '')

        rxn = model.reactions.get_by_id(reaction_id)
        genes = rxn.gene_reaction_rule

        genes_for_reaction.append(genes)

    genes_string = ' '.join(genes_for_reaction)

    list_of_genes.append(genes_string)

# add list of genes to dataframe
central_carbon_df['genes'] = list_of_genes

# save dataframe
central_carbon_df.to_csv('../results/central_carbon_genes/central_carbon_genes.csv', index=False)

central_carbon_df


,pathway,compartment,Equation,reaction_ids,genes
0,substrate_uptake,cytosol,Oleic Acid -> 9 AC,OCDCEAt,
0,substrate_uptake,cytosol,Glycerol -> GLYC,reverse_GLYCt,YALI0F00462g
0,substrate_uptake,cytosol,Glucose + ATP -> G6P,reverse_EX_glc(e),YALI0D01111g or YALI0D18876g or YALI0D00132g o...
1,emp,cytosol,G6P <-> F6P,PGI,YALI0F07711g
2,emp,cytosol,F6P + ATP <-> FBP,PFK or reverse_FBP,YALI0D16357g YALI0A15972g
...,...,...,...,...,...
74,transport,extracellular,CO2 -> CO2_EX,reverse_CO2t,
75,biomass_formation,cytosol,0.247*ALA + 0.146*ARG + 0.1165*ASN + 0.165*ASP...,biomass_C,
76,energy_metabolism,cytosol,ATP -> ATP_maintenance,ATPM,
77,energy_metabolism,cytosol,NADH -> NADPH,NaN,


In [5]:
model.reactions.get_by_id('OCDCEAt')

Reaction identifier,OCDCEAt
Name,Octadecenoate n C181 transport in via uniport
Memory address,0x136ef72b0
Stoichiometry,ocdcea[e] --> ocdcea[c] octadecenoate__n_C181_ --> octadecenoate__n_C181_
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
for r in model.metabolites.get_by_id('ocdcea[e]').reactions:
    display(r)

Reaction identifier,OCDCEAt
Name,Octadecenoate n C181 transport in via uniport
Memory address,0x136ef72b0
Stoichiometry,ocdcea[e] --> ocdcea[c] octadecenoate__n_C181_ --> octadecenoate__n_C181_
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,exlip
Name,extracelular lipases TAG
Memory address,0x137101fc0
Stoichiometry,triglyc_SC[e] --> 0.02 dca[e] + 0.06 ddca[e] + 0.03 glyc[e] + 0.27 hdca[e] + 0.17 hdcea[e] + 0.05 ocdca[e] + 0.24 ocdcea[e] + 0.09 ocdcya[e] + 0.1 ttdca[e] triacylglycerol --> 0.02 Decanoate__n_C100_ + 0.06 Dodecanoate__n_C120_ + 0.03 Glycerol + 0.27 Hexadecanoate__n_C160_ + 0.17 Hexadecenoate__n_C161_ + 0.05 octadecanoate__n_C180_ + 0.24...
GPR,YALI0A20350g
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,EX_ocdcea(e)
Name,octadecenoate n C181 exchange
Memory address,0x136c37370
Stoichiometry,ocdcea[e] --> octadecenoate__n_C181_ -->
GPR,
Lower bound,0.0
Upper bound,1000.0
